In [1]:
import numpy as np
import pandas as pd 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import torch as to
import torch.nn as nn 
import torch.nn.functional as func
from torch.utils import data
from torch.utils.data import DataLoader
import random 
import time 
import torch.nn.utils.rnn as rnnutils
from sklearn.model_selection import train_test_split
import torchviz
from torchsummary import summary
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score 
from itertools import tee
import hashlib
from torch.optim.lr_scheduler import StepLR
import gc

Using TensorFlow backend.



### Info 
This notebook is a continuation of the 'baseline.ipynb' nootebook. 

### Setup 
+> Added balancing of the data-set. Balancing is done in a way that samples are added until the less frequent class isn't same as more frequent class. Samples are added squentialy, not randomly, thus ensuring that the differenece of the most frequent sample and least frequent sample in the data-set is one.

+> f1 is now macro

+> batchnormalization was added befor convolution and removed :D 

+> we are currently stopping the training if f1 is not changing (max_constant_f1 denotes the max number of epochs that the f1 is not changing)

+> additonal data was added to the results dataframe (precision and recall for each class)

+> to ease the strain on memory this code has all input data is represented as indices, thus it runs a bit slower because it has to access the input dataframe to attain input vectors

+> original dataframe is preprocesed and deleted so that memory is freed 

+> now you can specify a gpu, ouput of all results, predictions and models (CONSTANT: 'MODELS_NAME')

In [2]:
#CELL METADATA
DATASET = '/home/mbosnjak/Datasets/embedded_comments_last_100_min_20com_per_auth_w_wc_10_200_no_mbti.csv'
FOLDS = '/home/mbosnjak/Datasets/comments_last_100_min_20com_per_auth_w_wc_10_200.csv.folds.csv'
AUTHORS = '/home/mbosnjak/Datasets/author_profiles.csv'

MODELS_NAME = 'baseline_take2_twin'
OUTPUT_DIR = f'mpr_{MODELS_NAME}/'
RUN_IDENTIFICATOR = 'perceivingIK'
RESULTS = OUTPUT_DIR + f'results_{MODELS_NAME}_{RUN_IDENTIFICATOR}.csv'


RELEVANT_DIFFERENCE = 0.0001
NUMBER_OF_CLASSES = 2
STATE = 156
no_of_folds = 5
cuda_device_index = 1

mbti_labels_1 = ['introverted', 'intuitive']
mbti_labels_2 = ['thinking', 'perceiving']
mbti_labels = [mbti_labels_2[1]] #+ mbti_labels_1
MODELS_PREFORMANCE_COLUMNS = ['val_f1', 'val_precision_0', 'val_precision_1', 'val_precision_macro' ,\
                              'val_recall_0', 'val_recall_1', 'val_recall_macro', \
                              'test_f1', 'test_precision_0', 'test_precision_1', 'test_precision_macro',\
                              'test_recall_0', 'test_recall_1', 'test_recall_macro',\
                              'epoch']
MODELS_PARAMETER_COLUMNS = ['mbti_trait', 'fold', 'learning_rate', 'batch_size',\
                           'kernels_count', 'sentences_count', 'decay_rate', 'decay_epoch', 'run_identificator']
MODELS_META_DATA_COLUMNS = ['models_name', 'hash_id']
to.manual_seed(STATE)

In [75]:
#CELL PARAMETERS
batch_sizes = [64]
learning_rates = [10**-5, 5*10**-6, 10**-6, 10**-7]
kernel_sizes = [12]
sentences_counts = [4]
n_epochs = 120
STEP_SIZE = 10
GAMMA = 0.90

use_GPU = True
#TODO: this is sentences per author not comments per author, quick fix is max size 200*100 (200 max sentences by comment * 100 comments)
comments_per_author = 20000
#comments_per_author = 20
cuda_device = to.device(cuda_device_index) if use_GPU else None
validation_p = .2
print_status_batch = 100
max_constant_f1 = 2 

In [4]:
def get_checkpoints_name(models_identifier):
    return OUTPUT_DIR+"save_"+MODELS_NAME+'_'+models_identifier + f'_{RUN_IDENTIFICATOR}_' + ".pt"

def get_predictions_file_name(models_identifier, target, fold):
    return OUTPUT_DIR + f'{target}_{fold}_{MODELS_NAME}_{RUN_IDENTIFICATOR}_{models_identifier}_predictions'

def delimiter():
    print('-'*23)

In [5]:
input_df = pd.read_csv(DATASET)
#input_df = pd.read_csv(DATASET, nrows = 2000) # TODO: remove nrows
folds_df = pd.read_csv(FOLDS, usecols =['author', 'fold'])
authors_profiles_df = pd.read_csv(AUTHORS)
authors_with_mbti = list(authors_profiles_df[authors_profiles_df.introverted.notnull()]['author'])

In [6]:
#TODO: add comment delimiter -> to recognize comments end and beginning
def merger(comments):
    npad = [(0, max(sentences_counts)), (0, 0)]
    carpet = comments[:comments_per_author].as_matrix(columns=comments.columns[1:1025])
    carpet_padded = np.pad(carpet, pad_width=npad, mode='constant', constant_values=0)
    #print(f'before {carpet.shape} - after {carpet_padded.shape}')
    return to.tensor(carpet_padded)

In [7]:
#Data preparation
data_df = input_df[input_df['author'].isin(authors_with_mbti)]
del input_df
present_authors = sorted(data_df['author'].unique())
authors_indices = dict(zip(present_authors , list(range(len(present_authors)))))
input_x_df = data_df.sort_values(by=['author', 'Unnamed: 0']).groupby(['author']).apply(merger).tolist()
del data_df

/home/mbosnjak/.conda/envs/pero/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [37]:
results = pd.DataFrame(data=None, columns=MODELS_META_DATA_COLUMNS+MODELS_PARAMETER_COLUMNS+MODELS_PREFORMANCE_COLUMNS)
for name in MODELS_PREFORMANCE_COLUMNS:
    results = results.astype({name: 'float64'})
for name in MODELS_META_DATA_COLUMNS:
    results = results.astype({name: 'object'})

In [42]:
gc.collect()

613

In [9]:
def set_hash(index):
    global results
    identifier = hashlib.md5(''.join([str(x) for x in results[MODELS_PARAMETER_COLUMNS].iloc[index]]).encode('utf-8')).hexdigest()
    results.at[index, 'hash_id'] = identifier
    return identifier

def create_new_model_input(mbti_trait, fold, learning_rate, batch_size,\
                           kernels_count, sentences_count):
    global results
    results = results.append({'models_name':MODELS_NAME, 'mbti_trait' : mbti_trait,\
                              'fold' : fold, 'learning_rate' : learning_rate,\
                              'batch_size' : batch_size, 'kernels_count' : kernels_count,\
                              'sentences_count' : sentences_count, 'decay_rate' : GAMMA,\
                              'decay_epoch' : STEP_SIZE, 'run_identificator' : RUN_IDENTIFICATOR,\
                             'hash_id': 'DUMMY_VALUE'}, ignore_index=True)
    return set_hash(len(results)-1)    

def update_models_val_results(models_identifier, val_f1, val_precision_0, val_precision_1, val_precision_m, val_recall_0, val_recall_1, val_recall_m, epoch):
    global results
    results.loc[results['hash_id'] == models_identifier, 'val_f1'] = val_f1
    results.loc[results['hash_id'] == models_identifier, 'val_precision_0'] = val_precision_0
    results.loc[results['hash_id'] == models_identifier, 'val_precision_1'] = val_precision_1
    results.loc[results['hash_id'] == models_identifier, 'val_precision_macro'] = val_precision_m
    results.loc[results['hash_id'] == models_identifier, 'val_recall_0'] = val_recall_0
    results.loc[results['hash_id'] == models_identifier, 'val_recall_1'] = val_recall_1
    results.loc[results['hash_id'] == models_identifier, 'val_recall_macro'] = val_recall_m
    results.loc[results['hash_id'] == models_identifier, 'epoch'] = epoch
    
def update_models_test_results(models_identifier, test_f1, test_precision_0, test_precision_1, precision_m, test_recall_0, test_recall_1, test_recall_m):
    global results
    results.loc[results['hash_id'] == models_identifier, 'test_f1'] = test_f1
    results.loc[results['hash_id'] == models_identifier, 'test_precision_0'] = test_precision_0
    results.loc[results['hash_id'] == models_identifier, 'test_precision_1'] = test_precision_1
    results.loc[results['hash_id'] == models_identifier, 'test_precision_macro'] = test_precision_m
    results.loc[results['hash_id'] == models_identifier, 'test_recall_0'] = test_recall_0
    results.loc[results['hash_id'] == models_identifier, 'test_recall_1'] = test_recall_1
    results.loc[results['hash_id'] == models_identifier, 'test_recall_macro'] = test_recall_m
    
def get_best_models_data(target, fold):
    return results.loc[results[(results.mbti_trait == target) & (results.fold == fold)]['val_f1'].idxmax()]
    

In [10]:
create_new_model_input('dfsfdsf',333,3,223333,1,13)
results.dtypes

models_name              object
hash_id                  object
mbti_trait               object
fold                     object
learning_rate            object
batch_size               object
kernels_count            object
sentences_count          object
decay_rate              float64
decay_epoch              object
run_identificator        object
val_f1                  float64
val_precision_0         float64
val_precision_1         float64
val_precision_macro     float64
val_recall_0            float64
val_recall_1            float64
val_recall_macro        float64
test_f1                 float64
test_precision_0        float64
test_precision_1        float64
test_precision_macro    float64
test_recall_0           float64
test_recall_1           float64
test_recall_macro       float64
epoch                   float64
dtype: object

In [47]:
results

,models_name,hash_id,mbti_trait,fold,learning_rate,batch_size,kernels_count,sentences_count,decay_rate,decay_epoch,...,val_recall_1,val_recall_macro,test_f1,test_precision_0,test_precision_1,test_precision_macro,test_recall_0,test_recall_1,test_recall_macro,epoch


### FLOW

In [76]:
def get_input_output(data_authors, target):
    output_prep = authors_profiles_df[authors_profiles_df['author'].isin(data_authors)][target].tolist()
    input_indices = [index for author, index in authors_indices.items() if author in data_authors]
    return input_indices, output_prep

def balance_binary_data(input_df, output_df):
    positive_indices = [index for index, element in enumerate(output_df) if element == 1.0]
    negative_indices = [index for index, element in enumerate(output_df) if element == 0.0]
    negative_count = len(negative_indices)
    positive_count = len(positive_indices)
    
    if(positive_count > negative_count):
        return create_balanced_data(positive_indices, negative_indices, input_df, output_df)
    elif(negative_count > positive_count):
        return create_balanced_data(negative_indices, positive_indices, input_df, output_df)
        
def create_balanced_data(more_frequent, less_frequent, input_df, output_df):
    more_frequent_count = len(more_frequent)
    less_frequent_count = len(less_frequent)
    constant_multiplyer = more_frequent_count // less_frequent_count
    remaining_additions = more_frequent_count % less_frequent_count
    balanced_indices = less_frequent*constant_multiplyer + less_frequent[:remaining_additions] + more_frequent
    output_df_balanced = [output_df[index] for index in balanced_indices]
    input_df_balanced = [input_df[index] for index in balanced_indices]
    return input_df_balanced, output_df_balanced
    
def data_preparation(target, fold):
    test_data_authors = folds_df[(folds_df['fold'] == fold) & (folds_df['author'].isin(present_authors))]['author'].tolist()
    train_data_authors = folds_df[(folds_df['fold'] != fold) & (folds_df['author'].isin(present_authors))]['author'].tolist()
    train_input_indices, train_output = get_input_output(train_data_authors, target)
    test_input_indices, test_output = get_input_output(test_data_authors, target)

    train_input_indices, val_input_indices, train_output, val_output = train_test_split(train_input_indices, train_output, test_size=validation_p, random_state=STATE)
    assert len(train_input_indices) == len(train_output)
    assert len(test_input_indices) == len(test_output)
    assert len(val_input_indices) == len(val_output)
    return train_input_indices, train_output, val_input_indices, val_output, test_input_indices, test_output

def chunks(l, n):
    return [l[i:i + n] for i in range(0, len(l), n)]  

def create_minibatches(data_X, data_y, minibatch_size, cuda_dev):
    for idx_list in chunks(range(len(data_X)), minibatch_size):
        data_X_indices = [data_X[index] for index in idx_list]
        data_y_idx = [int(data_y[index]) for index in idx_list]
        
        minibatch_X = [input_x_df[index] for index in data_X_indices]
        minibatch_X = rnnutils.pad_sequence(minibatch_X, batch_first=True, padding_value = 0) 
        minibatch_X = minibatch_X.unsqueeze(1)
        minibatch_y = to.tensor(data_y_idx)
        if cuda_dev is not None:
            minibatch_X = minibatch_X#.to(device=cuda_dev, dtype=to.float)
            minibatch_y = minibatch_y.to(device=cuda_dev, dtype=to.long)

        yield((minibatch_X, minibatch_y))   


In [77]:
class ConvoCarpet(nn.Module):
    def __init__ (self, embedding_size = 1024, kernels_count=64, sentences_count=2):
        super(ConvoCarpet, self).__init__()
        self.batch_norm = nn.BatchNorm2d(1)
        self.conv_layer = nn.Conv2d(1, kernels_count, [sentences_count, embedding_size])
        self.pool_layer = nn.AdaptiveMaxPool2d((1, None))
        self.fc_layer = nn.Linear(kernels_count, NUMBER_OF_CLASSES)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input_batch):
        #normalized_input = self.batch_norm(input_batch)
        conv_output = func.relu(self.conv_layer(input_batch))
        maxpool_output = self.pool_layer(conv_output)
        maxpool_output = maxpool_output.flatten(start_dim=1)
        linear_output = self.fc_layer(maxpool_output)
        output = self.softmax(linear_output)
        #return func.log_softmax(linear_output)
        return output


In [80]:
def calcMeasures(total_logits, total_true, threshold):
    total_preds = []
    for minibatch_logits in total_logits:
        minibatch_preds = minibatch_logits[:,1] > threshold
        total_preds.append(minibatch_preds)
    total_preds = to.cat(total_preds)
    total_true = to.cat(total_true)
    F1 = f1_score(total_true.cpu(), total_preds.cpu(), pos_label = 1, average='macro')
    precision_m = precision_score(total_true.cpu(), total_preds.cpu(), average='macro')
    precision_1 = precision_score(total_true.cpu(), total_preds.cpu(), pos_label = 1)
    precision_0 = precision_score(total_true.cpu(), total_preds.cpu(), pos_label = 0)
    recall_m = recall_score(total_true.cpu(), total_preds.cpu(), average='macro')
    recall_1 = recall_score(total_true.cpu(), total_preds.cpu(), pos_label = 1)
    recall_0 = recall_score(total_true.cpu(), total_preds.cpu(), pos_label = 0)
    return F1, precision_0, precision_1, precision_m, recall_0, recall_1, recall_m

def apply_model(model, loss_func, data_loader):
    model.eval()
    total_loss = 0
    predicted = []
    true_output = []
    minibatch_loss = 0
    with to.no_grad():
        n_batch = 0
        for inputs, labels in data_loader:
            n_batch += 1
            minibatch_logits = model(inputs.to(device=cuda_device, dtype=to.float))
            minibatch_loss = loss_func(minibatch_logits, labels)
            
            total_loss += minibatch_loss
            predicted.append(minibatch_logits)
            true_output.append(labels)
            
    return total_loss, predicted, true_output

def optimize_models_params(trait, fold, train_indices, output, val_input_indices, val_output):
    print('Start training ...')
    
    all_sum = len(output)
    positive = sum(output)
    negative = all_sum - positive
    print(f'Postive: {positive}: Negative: {negative}')
    loss_weights = to.tensor([negative, positive])
    loss_weights = loss_weights.to(device=cuda_device, dtype=to.float)
    loss = nn.CrossEntropyLoss(weight=loss_weights)
    
    train_input_indices, train_output = balance_binary_data(train_indices, output)
    
    for batch_size in batch_sizes:
        for kernel_size in kernel_sizes:
            for sent_size in sentences_counts:
                for learning_rate in learning_rates:                
                    
                    train_loader = create_minibatches(train_input_indices, train_output, batch_size, cuda_device)
                    val_loader = create_minibatches(val_input_indices, val_output, batch_size, cuda_device)
                    
                    delimiter()
                    delimiter()
                    print(f'Starting training for BS: {batch_size} KS: {kernel_size} SS: {sent_size} LR: {learning_rate}')
                    delimiter()
                    
                    models_identifier = create_new_model_input(trait, fold, learning_rate, batch_size, kernel_size, sent_size)
                    
                    model = ConvoCarpet(kernels_count=kernel_size, sentences_count=sent_size)
                    if use_GPU:
                          model.to(cuda_device)

                    optimizer = to.optim.Adam(model.parameters(), learning_rate, amsgrad = True)
                    
                    exp_lr_scheduler = StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
                    
                    train_model(train_loader, val_loader, models_identifier, model, loss, optimizer, exp_lr_scheduler)
                    results.to_csv(RESULTS)
    

def train_model(train_loader, val_loader, models_identifier, model, loss, optimizer, exp_lr_scheduler):
    best_f1_val = 0
    last_f1 = 0
    counter = 0
    
    for epoch in range(0, n_epochs):
        epoch_start = time.time()
        model.train()
        total_loss = 0
        n_batch = 0
        
        predicted = []
        true_output = []
        
        train_loader, train_loader_backup = tee(train_loader)
        for inputs, labels in train_loader_backup:

            n_batch += 1
            #print(f"Current batch is {n_batch}")

            model.zero_grad()

            logits = model(inputs.to(device=cuda_device, dtype=to.float))
            minibatch_loss = loss(logits, labels)
            total_loss += minibatch_loss
            minibatch_loss.backward()
            optimizer.step()
            predicted.append(logits)
            true_output.append(labels)
            
            if(n_batch % print_status_batch == 0):
                
                delimiter()               
                #preds = logits[:,1] > 0.5
                #train_f1_score = f1_score(labels.cpu().detach().numpy(), preds.cpu().detach().numpy(), pos_label = 1, average='macro')
                print(f"Train - Epoch {epoch} - batch {n_batch}, batch loss is {minibatch_loss:.6f}")
                delimiter()

        
        
        delimiter()
        exp_lr_scheduler.step()
        val_loader, val_loader_backup = tee(val_loader)
        val_loss, val_logits, true_val = apply_model(model, loss, val_loader_backup)
        val_f1_score, val_precision_0, val_precision_1, val_precision_m, val_recall_0, val_recall_1, val_recall_m = calcMeasures(val_logits, true_val, 0.5)
        train_f1_score, _, _, _, _, _,_ = calcMeasures(predicted, true_output, 0.5)
        
        print(f'Epoch {epoch} end: {time.time()-epoch_start}, F1 is: {train_f1_score}')
        print(f'Validation loss: {val_loss:.7f} - F1 score: {val_f1_score:.7f}')
        print(f'0 class -> precision: {val_precision_0:.7f} - recall: {val_recall_0:.7f}')
        print(f'1 class -> precision: {val_precision_1:.7f} - recall: {val_recall_1:.7f}')
        print(f'precision: {val_precision_m:.7f} - recall: {val_recall_m:.7f} - MACRO')
        delimiter()
        
        if (abs(val_f1_score - last_f1) <= RELEVANT_DIFFERENCE):
            counter += 1
        else:
            counter = 0
        
        last_f1 = val_f1_score
        
        if(counter >= max_constant_f1):
            return
        
        if (val_f1_score > best_f1_val):
            update_models_val_results(models_identifier, val_f1_score, val_precision_0, val_precision_1, val_precision_m, val_recall_0, val_precision_1, val_recall_m, epoch)
            best_f1_val = val_f1_score
        # save a checkpoint
            best_checkpoint_filename = get_checkpoints_name(models_identifier)
            to.save({
              "models_identifier" : models_identifier, 
              "model_state_dict" : model.state_dict(),
              "optimizer_state_dict" : optimizer.state_dict()
            }, best_checkpoint_filename)
    

In [79]:
for target in mbti_labels:
    print(f"Creating model for target: {target}")
    for fold in range(0, no_of_folds):
        
        print(f'Processing fold {fold}...')
        
        train_input_indices, train_output, val_input_indices, val_output, test_input_indices, test_output = data_preparation(target, fold)
        
        optimize_models_params(target, fold, train_input_indices, train_output, val_input_indices, val_output)
        
        
        best_models = get_best_models_data(target, fold)
        #Recreate model
        models_identifier = best_models['hash_id']
        model = ConvoCarpet(kernels_count=best_models['kernels_count'] , sentences_count=best_models['sentences_count'])
        if use_GPU:
              model.to(cuda_device)

        checkpoint = to.load(get_checkpoints_name(models_identifier))
        model.load_state_dict(checkpoint["model_state_dict"])
        loss = nn.CrossEntropyLoss()

        print("."*15)
        print("."*15)
        
        print(f'Apply best model to test for {target} on fold {fold}.')
        test_loader = create_minibatches(test_input_indices, test_output, 1, cuda_device)
        test_loss, test_logits, test_true = apply_model(model, loss, test_loader)
        test_f1_score, test_precision_0, test_precision_1, test_precision_m, test_recall_0, test_recall_1, test_recall_m = calcMeasures(test_logits, test_true, 0.5)
        print(f'Test loss: {test_loss:.5f} - F1 score: {test_f1_score:.7f} ')
        print(f'0 class -> precision: {test_precision_0:.7f} - recall: {test_recall_0:.7f}')
        print(f'1 class -> precision: {test_precision_1:.7f} - recall: {test_recall_1:.7f}')
        print(f'precision: {test_precision_m:.7f} - recall: {test_recall_m:.7f} - MACRO')
              
        update_models_test_results(models_identifier, test_f1_score, test_precision_0, test_precision_1, test_precision_m, test_recall_0, test_recall_1, test_recall_m)
        with open(get_predictions_file_name(models_identifier, target, fold), 'w+') as f:
            for pre_l, true_l in zip(test_logits, test_true):
                f.write(f'{pre_l.cpu()[0][0]}, {pre_l.cpu()[0][1]}, {true_l.cpu().tolist()[0]}\n')
        results.to_csv(RESULTS)
        
        ##Save best models data
        
        print(f"+++ Finished with training and testing model for {target} on fold {fold}. +++")
        
        print("."*15)
        print("."*15)

Creating model for target: perceiving
Processing fold 0...
Start training ...
Postive: 3341.0: Negative: 2311.0
-----------------------
-----------------------
Starting training for BS: 64 KS: 12 SS: 4 LR: 1e-05
-----------------------
-----------------------
Train - Epoch 0 - batch 100, batch loss is 0.651243, F1 is: 0.47540983606557374
-----------------------
-----------------------
Epoch 0 end: 6.960965633392334
Validation loss: 15.4964991 - F1 score: 0.3859269
0 class -> precision: 0.4210526 - recall: 0.0140598
1 class -> precision: 0.5978495 - recall: 0.9869822
precision: 0.5094510 - recall: 0.5005210 - MACRO
-----------------------
-----------------------
Train - Epoch 1 - batch 100, batch loss is 0.646857, F1 is: 0.488
-----------------------
-----------------------
Epoch 1 end: 3.398153305053711
Validation loss: 15.4684019 - F1 score: 0.3849810
0 class -> precision: 0.4666667 - recall: 0.0123023
1 class -> precision: 0.5982845 - recall: 0.9905325
precision: 0.5324756 - recall: 

-----------------------
Train - Epoch 20 - batch 100, batch loss is 0.632690, F1 is: 0.47107438016528924
-----------------------
-----------------------
Epoch 20 end: 3.9801454544067383
Validation loss: 15.4038391 - F1 score: 0.4149433
0 class -> precision: 0.4827586 - recall: 0.0492091
1 class -> precision: 0.6010324 - recall: 0.9644970
precision: 0.5418955 - recall: 0.5068531 - MACRO
-----------------------
-----------------------
Train - Epoch 21 - batch 100, batch loss is 0.633632, F1 is: 0.47107438016528924
-----------------------
-----------------------
Epoch 21 end: 3.906944513320923
Validation loss: 15.4099560 - F1 score: 0.4185404
0 class -> precision: 0.4769231 - recall: 0.0544815
1 class -> precision: 0.6011861 - recall: 0.9597633
precision: 0.5390546 - recall: 0.5071224 - MACRO
-----------------------
-----------------------
Train - Epoch 22 - batch 100, batch loss is 0.634175, F1 is: 0.4666666666666667
-----------------------
-----------------------
Epoch 22 end: 3.8818879

-----------------------
Train - Epoch 40 - batch 100, batch loss is 0.636196, F1 is: 0.46218487394957986
-----------------------
-----------------------
Epoch 40 end: 3.8765738010406494
Validation loss: 15.4431286 - F1 score: 0.4907114
0 class -> precision: 0.5348837 - recall: 0.1616872
1 class -> precision: 0.6159420 - recall: 0.9053254
precision: 0.5754129 - recall: 0.5335063 - MACRO
-----------------------
-----------------------
Train - Epoch 41 - batch 100, batch loss is 0.636941, F1 is: 0.46218487394957986
-----------------------
-----------------------
Epoch 41 end: 3.887587547302246
Validation loss: 15.4483747 - F1 score: 0.4910531
0 class -> precision: 0.5251397 - recall: 0.1652021
1 class -> precision: 0.6153846 - recall: 0.8994083
precision: 0.5702621 - recall: 0.5323052 - MACRO
-----------------------
-----------------------
Train - Epoch 42 - batch 100, batch loss is 0.637383, F1 is: 0.46218487394957986
-----------------------
-----------------------
Epoch 42 end: 3.752985

-----------------------
Train - Epoch 60 - batch 100, batch loss is 0.638196, F1 is: 0.4576271186440678
-----------------------
-----------------------
Epoch 60 end: 4.030089855194092
Validation loss: 15.4812260 - F1 score: 0.5271646
0 class -> precision: 0.5454545 - recall: 0.2319859
1 class -> precision: 0.6271331 - recall: 0.8698225
precision: 0.5862938 - recall: 0.5509042 - MACRO
-----------------------
-----------------------
Train - Epoch 61 - batch 100, batch loss is 0.638688, F1 is: 0.4576271186440678
-----------------------
-----------------------
Epoch 61 end: 3.886126756668091
Validation loss: 15.4851513 - F1 score: 0.5266463
0 class -> precision: 0.5335968 - recall: 0.2372583
1 class -> precision: 0.6261843 - recall: 0.8603550
precision: 0.5798906 - recall: 0.5488067 - MACRO
-----------------------
-----------------------
Train - Epoch 62 - batch 100, batch loss is 0.638999, F1 is: 0.4576271186440678
-----------------------
-----------------------
Epoch 62 end: 3.9911324977

-----------------------
Train - Epoch 10 - batch 100, batch loss is 0.684370, F1 is: 0.39622641509433965
-----------------------
-----------------------
Epoch 10 end: 3.793851375579834
Validation loss: 15.8229046 - F1 score: 0.5258470
0 class -> precision: 0.4536585 - recall: 0.3268893
1 class -> precision: 0.6185259 - recall: 0.7349112
precision: 0.5360922 - recall: 0.5309003 - MACRO
-----------------------
-----------------------
Train - Epoch 11 - batch 100, batch loss is 0.681401, F1 is: 0.39622641509433965
-----------------------
-----------------------
Epoch 11 end: 3.7404611110687256
Validation loss: 15.7997456 - F1 score: 0.5225625
0 class -> precision: 0.4626039 - recall: 0.2934974
1 class -> precision: 0.6182336 - recall: 0.7704142
precision: 0.5404187 - recall: 0.5319558 - MACRO
-----------------------
-----------------------
Train - Epoch 12 - batch 100, batch loss is 0.678738, F1 is: 0.4074074074074074
-----------------------
-----------------------
Epoch 12 end: 3.6902368

...

Epoch 2 end: 3.9164469242095947
Validation loss: 18.5924129 - F1 score: 0.2968285
0 class -> precision: 0.4221283 - recall: 1.0000000
1 class -> precision: 0.0000000 - recall: 0.0000000
precision: 0.2110641 - recall: 0.5000000 - MACRO
-----------------------
...............
...............
Apply best model to test for perceiving on fold 3.
Test loss: 1212.63989 - F1 score: 0.5227043 
0 class -> precision: 0.4705882 - recall: 0.2979310
1 class -> precision: 0.6114504 - recall: 0.7672414
precision: 0.5410193 - recall: 0.5325862 - MACRO
+++ Finished with training and testing model for perceiving on fold 3. +++
...............
...............
Processing fold 4...
Start training ...
Postive: 3310.0: Negative: 2362.0
-----------------------
-----------------------
Starting training for BS: 64 KS: 12 SS: 4 LR: 1e-05
-----------------------
-----------------------
Train - Epoch 0 - batch 100, batch loss is 0.666688, F1 is: 0.47540983606557374
-----------------------
-----------------------
Epo

-----------------------
Train - Epoch 6 - batch 100, batch loss is 0.699504, F1 is: 0.2808988764044944
-----------------------
-----------------------
Epoch 6 end: 3.674663543701172
Validation loss: 15.9797411 - F1 score: 0.4940036
0 class -> precision: 0.4148061 - recall: 0.6160558
1 class -> precision: 0.6126761 - recall: 0.4113475
precision: 0.5137411 - recall: 0.5137017 - MACRO
-----------------------
-----------------------
Train - Epoch 7 - batch 100, batch loss is 0.698326, F1 is: 0.2808988764044944
-----------------------
-----------------------
Epoch 7 end: 3.5101451873779297
Validation loss: 15.9699850 - F1 score: 0.4925276
0 class -> precision: 0.4113390 - recall: 0.5951134
1 class -> precision: 0.6067797 - recall: 0.4231678
precision: 0.5090593 - recall: 0.5091406 - MACRO
-----------------------
-----------------------
Train - Epoch 8 - batch 100, batch loss is 0.697199, F1 is: 0.28888888888888886
-----------------------
-----------------------
Epoch 8 end: 3.95574688911438

-----------------------
Train - Epoch 26 - batch 100, batch loss is 0.684875, F1 is: 0.37254901960784315
-----------------------
-----------------------
Epoch 26 end: 4.116473197937012
Validation loss: 15.8647060 - F1 score: 0.5169510
0 class -> precision: 0.4235294 - recall: 0.4397906
1 class -> precision: 0.6104369 - recall: 0.5945626
precision: 0.5169832 - recall: 0.5171766 - MACRO
-----------------------
-----------------------
Train - Epoch 27 - batch 100, batch loss is 0.684467, F1 is: 0.3786407766990291
-----------------------
-----------------------
Epoch 27 end: 4.325395345687866
Validation loss: 15.8614798 - F1 score: 0.5167959
0 class -> precision: 0.4234694 - recall: 0.4345550
1 class -> precision: 0.6101083 - recall: 0.5992908
precision: 0.5167888 - recall: 0.5169229 - MACRO
-----------------------
-----------------------
Train - Epoch 28 - batch 100, batch loss is 0.684068, F1 is: 0.3786407766990291
-----------------------
-----------------------
Epoch 28 end: 4.671375751

-----------------------
Train - Epoch 46 - batch 100, batch loss is 0.679248, F1 is: 0.40186915887850466
-----------------------
-----------------------
Epoch 46 end: 4.146141052246094
Validation loss: 15.8244047 - F1 score: 0.5145091
0 class -> precision: 0.4254032 - recall: 0.3682373
1 class -> precision: 0.6078007 - recall: 0.6631206
precision: 0.5166019 - recall: 0.5156790 - MACRO
-----------------------
-----------------------
Train - Epoch 47 - batch 100, batch loss is 0.679081, F1 is: 0.40186915887850466
-----------------------
-----------------------
Epoch 47 end: 4.125404357910156
Validation loss: 15.8231249 - F1 score: 0.5150797
0 class -> precision: 0.4262626 - recall: 0.3682373
1 class -> precision: 0.6082251 - recall: 0.6643026
precision: 0.5172439 - recall: 0.5162700 - MACRO
-----------------------
-----------------------
Train - Epoch 48 - batch 100, batch loss is 0.678917, F1 is: 0.40186915887850466
-----------------------
-----------------------
Epoch 48 end: 4.6608402

-----------------------
Train - Epoch 66 - batch 100, batch loss is 0.676777, F1 is: 0.42857142857142855
-----------------------
-----------------------
Epoch 66 end: 4.871391773223877
Validation loss: 15.8077526 - F1 score: 0.5175896
0 class -> precision: 0.4312500 - recall: 0.3612565
1 class -> precision: 0.6102236 - recall: 0.6773050
precision: 0.5207368 - recall: 0.5192808 - MACRO
-----------------------
-----------------------
Train - Epoch 67 - batch 100, batch loss is 0.676703, F1 is: 0.42857142857142855
-----------------------
-----------------------
Epoch 67 end: 4.200189828872681
Validation loss: 15.8072081 - F1 score: 0.5175896
0 class -> precision: 0.4312500 - recall: 0.3612565
1 class -> precision: 0.6102236 - recall: 0.6773050
precision: 0.5207368 - recall: 0.5192808 - MACRO
-----------------------
-----------------------
Train - Epoch 68 - batch 100, batch loss is 0.676629, F1 is: 0.42857142857142855
-----------------------
-----------------------
Epoch 68 end: 4.3085787

In [73]:
gc.collect()

1387